In [1]:
# instalar bigquery 
!pip install google-cloud-bigquery -q

In [2]:
# cargamos las librerías
from google.cloud import bigquery
from google.oauth2 import service_account

In [ ]:
creds = service_account.Credentials.from_service_account_file('/content/thebridgept0521-00ccc9a47591.json')

In [ ]:
# En caso de trabajar con el token json
client = bigquery.Client(credentials=creds, project=proj_id)

***
### Alternativa con Auth de Google

In [3]:
# Alternativa con Google Oauth de Google Colab
from google.colab import auth
auth.authenticate_user()

In [4]:
# introducimos el projectID
proj_id = 'thebridgept0521'

In [5]:
# En caso de trabajar con el token
client = bigquery.Client(project=proj_id)

***

In [12]:
# Probamos Gcp-BQ
query = (
'SELECT name '
'FROM `bigquery-public-data.usa_names.usa_1910_2013` '
'WHERE state = "TX" '
'LIMIT 100 '
)

In [13]:
# Ejecutaremos el job de BigQuery
query_job = client.query(query) # esta es la llamada a la API

In [14]:
df_1 = query_job.to_dataframe()
df_1

,name
0,Frances
1,Alice
2,Beatrice
3,Ella
4,Gertrude
...,...
95,Rosa
96,Lucy
97,Nora
98,Nettie


***

## Predicción de compra desde Google BigQuery

In [15]:
query = """
#standardSQL
WITH visitors AS(
SELECT
COUNT(DISTINCT fullVisitorId) AS total_visitors
FROM `data-to-insights.ecommerce.web_analytics`
),
purchasers AS(
SELECT
COUNT(DISTINCT fullVisitorId) AS total_purchasers
FROM `data-to-insights.ecommerce.web_analytics`
WHERE totals.transactions IS NOT NULL
)
SELECT
 total_visitors,
 total_purchasers,
 total_purchasers / total_visitors AS conversion_rate
FROM visitors, purchasers

"""

In [16]:
query_job = client.query(query)
df_2 = query_job.to_dataframe()
df_2

,total_visitors,total_purchasers,conversion_rate
0,741721,20015,0.026985


In [25]:
query = """
SELECT
  p.v2ProductName,
  p.v2ProductCategory,
  SUM(p.productQuantity) AS units_sold,
  ROUND(SUM(p.localProductRevenue/1000000),2) AS revenue
FROM `data-to-insights.ecommerce.web_analytics`,
UNNEST(hits) AS h,
UNNEST(h.product) AS p
WHERE date BETWEEN '20160101' AND '20161231' 
GROUP BY 1, 2
ORDER BY revenue DESC
LIMIT 10;

"""

In [26]:
query_job = client.query(query)
df = query_job.to_dataframe()
df

,v2ProductName,v2ProductCategory,units_sold,revenue
0,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,Nest-USA,7969,370141.09
1,Nest® Cam Outdoor Security Camera - USA,Nest-USA,6570,249730.11
2,Nest® Cam Indoor Security Camera - USA,Nest-USA,4763,174158.36
3,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,2090,55584.32
4,Nest® Protect Smoke + CO White Wired Alarm-USA,Nest-USA,2002,50799.52
5,Nest® Learning Thermostat 3rd Gen - CA - Stain...,Nest-Canada,383,17432.17
6,26 oz Double Wall Insulated Bottle,Drinkware,2153,16138.09
7,Google 22 oz Water Bottle,Drinkware,17331,14431.35
8,Google Men's Zip Hoodie,(not set),884,12859.26
9,Google Metallic Notebook Set,(not set),7028,10512.88


In [23]:
query = """
# visitors who bought on a return visit (could have bought on first as well
WITH all_visitor_stats AS (
SELECT
  fullvisitorid, # 741,721 unique visitors
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid
)
SELECT
  COUNT(DISTINCT fullvisitorid) AS total_visitors,
  will_buy_on_return_visit
FROM all_visitor_stats
GROUP BY will_buy_on_return_visit

"""

In [24]:
query_job = client.query(query)
df = query_job.to_dataframe()
df

,total_visitors,will_buy_on_return_visit
0,729848,0
1,11873,1


In [28]:
query = """
SELECT
  * EXCEPT(fullVisitorId)
FROM
  # features
  (SELECT
    fullVisitorId,
    IFNULL(totals.bounces, 0) AS bounces,
    IFNULL(totals.timeOnSite, 0) AS time_on_site
  FROM
    `data-to-insights.ecommerce.web_analytics`
  WHERE
    totals.newVisits = 1)
  JOIN
  (SELECT
    fullvisitorid,
    IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM
      `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid)
  USING (fullVisitorId)
ORDER BY time_on_site DESC
LIMIT 10;
"""

In [29]:
query_job = client.query(query)
df = query_job.to_dataframe()
df

,bounces,time_on_site,will_buy_on_return_visit
0,0,15047,0
1,0,12136,0
2,0,11201,0
3,0,10046,0
4,0,9974,0
5,0,9564,0
6,0,9520,0
7,0,9275,1
8,0,9138,0
9,0,8872,0


## Creación del modelo BigQuery ML

In [34]:
query = """
CREATE OR REPLACE MODEL `ecommerce.classification_model_mrusso`
OPTIONS
(
model_type='logistic_reg',
labels = ['will_buy_on_return_visit']
)
AS
#standardSQL
SELECT
  * EXCEPT(fullVisitorId)
FROM
  # features
  (SELECT
    fullVisitorId,
    IFNULL(totals.bounces, 0) AS bounces,
    IFNULL(totals.timeOnSite, 0) AS time_on_site
  FROM
    `data-to-insights.ecommerce.web_analytics`
  WHERE
    totals.newVisits = 1
    AND date BETWEEN '20160801' AND '20170430') # train on first 9 months
  JOIN
  (SELECT
    fullvisitorid,
    IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM
      `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid)
  USING (fullVisitorId)
;
"""

In [35]:
query_job = client.query(query)
df = query_job.to_dataframe()
df

""
